<a href="https://colab.research.google.com/github/ajeevansiva/stock_news_summarizer/blob/main/crypto_stocks_news_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install and Import Baseline Dependencies

In [1]:
!pip install transformers

In [2]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

## 2. Setup Summarization Model

In [3]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at human-centered-summarization/financial-summarization-pegasus and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 3. Summarize a Single Article

In [4]:
url = "https://finance.yahoo.com/news/binances-exit-nigeria-means-p2p-193018323.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [5]:
paragraphs

[<p id="message-1">Thank you for your patience.</p>,
 <p id="message-2">Our engineers are working quickly to resolve the issue.</p>]

In [6]:
r.text

'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  }\n  h1 {\n      font-size: 42px;\n      font-weight: 400;\n      color: #400090;\n  }\n  p {\n      color: #1A1A1A;\n  }\n  #message-1 {\n      font-weight: bold;\n      margin: 

In [7]:
paragraphs[0].text

'Thank you for your patience.'

In [8]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [9]:
ARTICLE

'Thank you for your patience. Our engineers are working quickly to resolve the issue.'

In [10]:
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt')
output = model.generate(input_ids, max_length=120, num_beams=5, early_stopping=True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [11]:
summary

'We are aware of the issue and are working to resolve it.'

In [12]:
output

tensor([[   0,  184,  127, 1906,  113,  109,  797,  111,  127,  375,  112, 4568,
          126,  107,    1]])

# 4. Building a News and Sentiment Pipeline

In [13]:
monitored_tickers = ['GME', 'TSLA', 'BTC']

In [14]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [15]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'GME': ['/?sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQOwgC',
  '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=nws&gbv=1&sei=Pg0FZoDvA-nIp84PwqKmiA0',
  '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUIBSgA',
  '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUIBygC',
  '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUICCgD',
  '/url?q=https://maps.google.com/maps%3Fq%3Dyahoo%2Bfinance%2BGME%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQiaAMCAkoBA&usg=AOvVaw0LtU_b1As1wCMrTUGBsfge',
  '/url?q=/search%3Fq%3Dyahoo%2Bfinance%2BGME%26sca_esv%3Df467e1d66e3ba319%26ie%3DUTF-8%26tbm%3Dshop%26source%3Dlnms%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdE

In [16]:
raw_urls.keys()

dict_keys(['GME', 'TSLA', 'BTC'])

In [17]:
raw_urls.values()

dict_values([['/?sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQOwgC', '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=nws&gbv=1&sei=Pg0FZoDvA-nIp84PwqKmiA0', '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUIBSgA', '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUIBygC', '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUICCgD', '/url?q=https://maps.google.com/maps%3Fq%3Dyahoo%2Bfinance%2BGME%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQiaAMCAkoBA&usg=AOvVaw0LtU_b1As1wCMrTUGBsfge', '/url?q=/search%3Fq%3Dyahoo%2Bfinance%2BGME%26sca_esv%3Df467e1d66e3ba319%26ie%3DUTF-8%26tbm%3Dshop%26source%3Dlnms%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQiaAMCA

In [18]:
raw_urls['GME']

['/?sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQOwgC',
 '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=nws&gbv=1&sei=Pg0FZoDvA-nIp84PwqKmiA0',
 '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUIBSgA',
 '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUIBygC',
 '/search?q=yahoo+finance+GME&sca_esv=f467e1d66e3ba319&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQ_AUICCgD',
 '/url?q=https://maps.google.com/maps%3Fq%3Dyahoo%2Bfinance%2BGME%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQiaAMCAkoBA&usg=AOvVaw0LtU_b1As1wCMrTUGBsfge',
 '/url?q=/search%3Fq%3Dyahoo%2Bfinance%2BGME%26sca_esv%3Df467e1d66e3ba319%26ie%3DUTF-8%26tbm%3Dshop%26source%3Dlnms%26ved%3D1t:200713%26ictx%3D111&opi=89978449&sa=U&ved=0ahUKEwiAhsGqqZaFAxVp5MkDHUKRCdEQiaAMCAooBQ&us

In [19]:
import re

In [20]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [21]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [22]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'GME': ['https://finance.yahoo.com/news/gamestop-gme-q4-earnings-revenues-212003206.html',
  'https://finance.yahoo.com/news/why-gamestop-gme-shares-trading-162132483.html',
  'https://finance.yahoo.com/video/gamestop-plunges-earnings-trump-media-140954471.html',
  'https://finance.yahoo.com/news/gamestop-reports-fourth-quarter-fiscal-200500541.html',
  'https://finance.yahoo.com/video/gamestop-shares-fall-missing-q4-210828603.html',
  'https://finance.yahoo.com/news/gamestop-nyse-gme-misses-q4-200940979.html',
  'https://finance.yahoo.com/news/gamestop-gme-q4-earnings-miss-143600702.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26tbm%3Dnws%26pccc%3D1',
  'https://finance.yahoo.com/news/earnings-watch-gamestop-gme-reports-184241519.html',
  'https://finance.yahoo.com/news/unveiling-gamestop-gme-value-really-155119864.html',
  'https://finance.yahoo.com/video/fedspeak-pce-print-gamestop-earnings-203144851.html'],
 'TSLA': ['https://finance.yahoo.com/news/why-tesla-s

In [23]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [24]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'GME': ['Thank you for your patience. Our engineers are working quickly to resolve the issue.',
  "Shares of video game retailer GameStop (NYSE:GME) fell 19.9% in the pre-market session after the company reported fourth-quarter results with revenue unfortunately falling short of analysts' expectations, driven by significant underperformance in its software segment ($465m of revenue vs. estimates of $670m). Because the company's software products have higher profit margins than its hardware products, it missed Wall Street's estimates on nearly every profitability metric. The company also didn't provide any guidance and said it would not host a conference call to discuss the results, which raises an eyebrow. Overall, this was a bad quarter for GameStop. The stock market overreacts to news, and big price drops can present good opportunities to buy high-quality stocks. Is now the time to buy GameStop? Access our full analysis report here, it's free. GameStop's shares are very volatile and

In [25]:
articles['TSLA']

['Shares of World No. 2 electric vehicle (EV) stock Tesla (NASDAQ: TSLA) jumped 5.2% through 10 a.m. ET Tuesday, despite Bernstein cutting the stock\'s price target to $120 per share. Bernstein cited soft demand for Tesla\'s EVs as its reason for lowering the price target, reducing predicted Tesla car sales, and maintaining its sell rating. So why is Tesla up despite this news? Because Italy. As Il Sole 24 reports, the electric car maker\xa0is negotiating with the Italian government to secure an Italian production site -- not for producing electric cars. Tesla wants Italy as home base in Europe for building electric trucks and vans. From Italy\'s perspective, this would bring a second automotive manufacturer to the country, joining Stellantis, which currently dominates. (Three Chinese companies are also negotiating.) But why would Tesla -- which already has a car plan in Germany -- want to build a new plant? Details aren\'t 100% clear, but if Tesla plans to introduce its electric Semi 

In [26]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [27]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'GME': ['We are aware of the issue and are working to resolve it.',
  "Revenue fell short of analysts' expectations, driven by significant underperformance.",
  'We are aware of the issue and are working to resolve it.',
  'Fiscal year 2023 net sales were $5.273 billion, compared to $5.927 billion in 2022.',
  'We are aware of the issue and are working to resolve it.',
  "Revenue fell 19.4% year on year to $1.79 billion, missed analysts' expectations.",
  'We are aware of the issue and are working to resolve it.',
  'Your information may be shared with third parties.',
  'Analysts are expecting the business to stay the course heading into the earnings.',
  'We are aware of the issue and are working to resolve it.',
  'We are aware of the issue and are working to resolve it.'],
 'TSLA': ['Italian newspaper says Tesla is in talks for a new plant.',
  'Tesla is down 31% this year and off 58% from its all-time high. Netflix is down 76% from its all-time high in November 2021',
  'Tesla, N

In [28]:
summaries['BTC']

['We are aware of the issue and are working to resolve it.',
 'Your information may be shared with third parties.',
 'We are aware of the issue and are working to resolve it.',
 'We are aware of the issue and are working to resolve it.',
 'We are aware of the issue and are working to resolve it.',
 'Prosecutors ask for 40 to 50 years; Bankman-Fried’s lawyers ask for six and a half years.',
 'Social trading platform Option2Trade (O2T) is attracting a diverse user base.',
 'We are aware of the issue and are working to resolve it.',
 'We are aware of the issue and are working to resolve it.',
 'We are aware of the issue and are working to resolve it.',
 'Tesla to offer US customers a free trial of self-drive technology. Bitcoin falls below $70,000 as crypto rally fades']

In [29]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [30]:
sentiment(summaries['BTC'])

[{'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'NEGATIVE', 'score': 0.9903545379638672},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'NEGATIVE', 'score': 0.9838812351226807},
 {'label': 'POSITIVE', 'score': 0.9984980821609497},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'POSITIVE', 'score': 0.9979088306427002},
 {'label': 'NEGATIVE', 'score': 0.9996228218078613}]

In [31]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'GME': [{'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9992107152938843},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9288461208343506},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9992019534111023},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9903545379638672},
  {'label': 'NEGATIVE', 'score': 0.99628084897995},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9979088306427002}],
 'TSLA': [{'label': 'NEGATIVE', 'score': 0.9174425005912781},
  {'label': 'NEGATIVE', 'score': 0.9994245767593384},
  {'label': 'POSITIVE', 'score': 0.9998431205749512},
  {'label': 'NEGATIVE', 'score': 0.9996353387832642},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9866054654121399},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'N

In [32]:
print(summaries['GME'][3], scores['GME'][3]['label'], scores['GME'][3]['score'])

Fiscal year 2023 net sales were $5.273 billion, compared to $5.927 billion in 2022. POSITIVE 0.9288461208343506


In [33]:
summaries

{'GME': ['We are aware of the issue and are working to resolve it.',
  "Revenue fell short of analysts' expectations, driven by significant underperformance.",
  'We are aware of the issue and are working to resolve it.',
  'Fiscal year 2023 net sales were $5.273 billion, compared to $5.927 billion in 2022.',
  'We are aware of the issue and are working to resolve it.',
  "Revenue fell 19.4% year on year to $1.79 billion, missed analysts' expectations.",
  'We are aware of the issue and are working to resolve it.',
  'Your information may be shared with third parties.',
  'Analysts are expecting the business to stay the course heading into the earnings.',
  'We are aware of the issue and are working to resolve it.',
  'We are aware of the issue and are working to resolve it.'],
 'TSLA': ['Italian newspaper says Tesla is in talks for a new plant.',
  'Tesla is down 31% this year and off 58% from its all-time high. Netflix is down 76% from its all-time high in November 2021',
  'Tesla, N

In [34]:
scores

{'GME': [{'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9992107152938843},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9288461208343506},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9992019534111023},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9903545379638672},
  {'label': 'NEGATIVE', 'score': 0.99628084897995},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9979088306427002}],
 'TSLA': [{'label': 'NEGATIVE', 'score': 0.9174425005912781},
  {'label': 'NEGATIVE', 'score': 0.9994245767593384},
  {'label': 'POSITIVE', 'score': 0.9998431205749512},
  {'label': 'NEGATIVE', 'score': 0.9996353387832642},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9866054654121399},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'N

In [35]:
cleaned_urls

{'GME': ['https://finance.yahoo.com/news/gamestop-gme-q4-earnings-revenues-212003206.html',
  'https://finance.yahoo.com/news/why-gamestop-gme-shares-trading-162132483.html',
  'https://finance.yahoo.com/video/gamestop-plunges-earnings-trump-media-140954471.html',
  'https://finance.yahoo.com/news/gamestop-reports-fourth-quarter-fiscal-200500541.html',
  'https://finance.yahoo.com/video/gamestop-shares-fall-missing-q4-210828603.html',
  'https://finance.yahoo.com/news/gamestop-nyse-gme-misses-q4-200940979.html',
  'https://finance.yahoo.com/news/gamestop-gme-q4-earnings-miss-143600702.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26tbm%3Dnws%26pccc%3D1',
  'https://finance.yahoo.com/news/earnings-watch-gamestop-gme-reports-184241519.html',
  'https://finance.yahoo.com/news/unveiling-gamestop-gme-value-really-155119864.html',
  'https://finance.yahoo.com/video/fedspeak-pce-print-gamestop-earnings-203144851.html'],
 'TSLA': ['https://finance.yahoo.com/news/why-tesla-s

In [36]:
range(len(summaries['GME']))

range(0, 11)

In [37]:
summaries['GME'][3]

'Fiscal year 2023 net sales were $5.273 billion, compared to $5.927 billion in 2022.'

In [38]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [39]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['GME',
  'We are aware of the issue and are working to resolve it.',
  'POSITIVE',
  0.9979088306427002,
  'https://finance.yahoo.com/news/gamestop-gme-q4-earnings-revenues-212003206.html'],
 ['GME',
  "Revenue fell short of analysts' expectations, driven by significant underperformance.",
  'NEGATIVE',
  0.9992107152938843,
  'https://finance.yahoo.com/news/why-gamestop-gme-shares-trading-162132483.html'],
 ['GME',
  'We are aware of the issue and are working to resolve it.',
  'POSITIVE',
  0.9979088306427002,
  'https://finance.yahoo.com/video/gamestop-plunges-earnings-trump-media-140954471.html'],
 ['GME',
  'Fiscal year 2023 net sales were $5.273 billion, compared to $5.927 billion in 2022.',
  'POSITIVE',
  0.9288461208343506,
  'https://finance.yahoo.com/news/gamestop-reports-fourth-quarter-fiscal-200500541.html'],
 ['GME',
  'We are aware of the issue and are working to resolve it.',
  'POSITIVE',
  0.9979088306427002,
  'https://finance.yahoo.com/video/gamestop-shares-fall-m

In [40]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])

In [41]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['GME',
  'We are aware of the issue and are working to resolve it.',
  'POSITIVE',
  0.9979088306427002,
  'https://finance.yahoo.com/news/gamestop-gme-q4-earnings-revenues-212003206.html'],
 ['GME',
  "Revenue fell short of analysts' expectations, driven by significant underperformance.",
  'NEGATIVE',
  0.9992107152938843,
  'https://finance.yahoo.com/news/why-gamestop-gme-shares-trading-162132483.html'],
 ['GME',
  'We are aware of the issue and are working to resolve it.',
  'POSITIVE',
  0.9979088306427002,
  'https://finance.yahoo.com/video/gamestop-plunges-earnings-trump-media-140954471.html'],
 ['GME',
  'Fiscal year 2023 net sales were $5.273 billion, compared to $5.927 billion in 2022.',
  'POSITIVE',
  0.9288461208343506,
  'https://finance.yahoo.com/news/gamestop-reports-fourth-quarter-fiscal-200500541.html'],
 ['GME',
  'We are aware of the issue and are working to resolve it.',
  'POSITIVE',
  0.9979088306427002,
  '

In [42]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)